In [1]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess

In [2]:
%run common.ipynb

In [3]:
train_df = pd.read_pickle("./train.pkl")
train_df.head()

,ari_time,ari_title,price
139,2010-06-13,ANZ Bank Seeking to Double Its Northeast Asian...,1
145,2010-06-13,Abu Dhabi Said to Invite Sulfur Granulation Pl...,1
47,2010-06-13,"Agility, Arabtec, Gulf Finance House, Sahara, ...",1
188,2010-06-13,"Air India, Boeing to Hold Talks Over Plane Del...",1
174,2010-06-13,"Al-Qaeda Gunmen Storm Iraqi Central Bank, Kill...",1


In [4]:
train_reduce_df = pd.read_pickle("./train_reduce.pkl")
train_reduce_df.head()

,ari_time,ari_title,price
0,2010-06-13,bank of new zealand say to start marketing nz$...,1
1,2010-06-14,banking regulator say forces driving china 's ...,1
2,2010-06-15,broad commits # % of wealth take buffett pledg...,0
3,2010-06-16,u.s. stocks trim losses south korea tell north...,1
4,2010-06-17,australian prosecutors get evidence from monta...,0


In [5]:
%%time
# del corpus
corpus = list(map(simple_preprocess, train_reduce_df)) # ['ari_title']
corpus += list(map(simple_preprocess, train_df['ari_title']))

print(corpus[0:10])

[['ari_time'], ['ari_title'], ['price'], ['anz', 'bank', 'seeking', 'to', 'double', 'its', 'northeast', 'asian', 'revenue', 'australian', 'reports'], ['abu', 'dhabi', 'said', 'to', 'invite', 'sulfur', 'granulation', 'plant', 'bids', 'for', 'shah', 'gas', 'field'], ['agility', 'arabtec', 'gulf', 'finance', 'house', 'sahara', 'zain', 'gulf', 'equity', 'preview'], ['air', 'india', 'boeing', 'to', 'hold', 'talks', 'over', 'plane', 'delivery', 'press', 'trust', 'reports'], ['al', 'qaeda', 'gunmen', 'storm', 'iraqi', 'central', 'bank', 'kill', 'wound', 'army', 'says'], ['algeria', 'slovenia', 'tied', 'at', 'halftime', 'of', 'world', 'cup', 'group', 'game'], ['asian', 'junk', 'bonds', 'risk', 'company', 'calls', 'amid', 'biggest', 'debt', 'rally', 'this', 'decade']]
CPU times: user 9.05 s, sys: 127 ms, total: 9.18 s
Wall time: 9.18 s


In [6]:
%%time

model = Word2Vec(corpus, size=EMBEDDING_DIM, iter=10)
model.save('word2vec.model')

CPU times: user 1min 27s, sys: 722 ms, total: 1min 27s
Wall time: 31.9 s


In [7]:
# [Optional] Calculate max length among all titles for checking, ref MAX_WORD_COUNT in constants.ipynb
max_word_count = np.max(list(map(len, corpus)))
print("Max word count is %d among %d titles." % (max_word_count, len(train_df['ari_title'])))

Max word count is 17 among 312217 titles.


In [8]:
# [Optional] Calculate number of unseen words for checking
words = {}
word_count = 0
unseen_count = 0
for wl in corpus:
    for word in wl:
        word_count += 1
        if word in words:
            continue
        else:
            words[word] = True
        if word not in model.wv.vocab:
            unseen_count += 1            
non_repeated_words = len(words)

print(model)
print(train_df.shape)
print(word_count, non_repeated_words, unseen_count)
print(unseen_count / non_repeated_words)

Word2Vec(vocab=22839, size=100, alpha=0.025)
(312217, 3)
2896872 61424 38585
0.6281746548580359
